In [ ]:
# pip install optimum[openvino,nncf] torchvision 
# pip install onnx
# pip install --upgrade protobuf numpy
# git clone https://github.com/huggingface/optimum-intel

# python -m pip install optimum
# pip install --upgrade --upgrade-strategy eager optimum[openvino] 

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("pszemraj/flan-t5-large-grammar-synthesis")
model = AutoModelForSeq2SeqLM.from_pretrained("pszemraj/flan-t5-large-grammar-synthesis")

def correct_grammar(text):
    input_ids = tokenizer(f"grammar: {text}", return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

text = "This sentences have bad grammer."
corrected_text = correct_grammar(text)
print(f"Original: {text}")
print(f"Corrected: {corrected_text}")

Original: This sentences have bad grammer.
Corrected: grammar: these sentences have bad grammar.


In [2]:
import os
save_directory = "./saved_grammar_model"

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'min_length': 8, 'num_beams': 2, 'no_repeat_ngram_size': 4}


Model and tokenizer saved to ./saved_grammar_model


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the saved model and tokenizer
save_directory = "./saved_grammar_model"
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [4]:
def correct_grammar(text):
    input_ids = tokenizer(f"grammar: {text}", return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True), input_ids

# Example usage
text = "bor"
corrected_text, input_ids = correct_grammar(text)
print(f"Original: {text}")
print(f"Corrected: {corrected_text}")

Original: bor
Corrected: grammar: oh boy.


In [1]:
from transformers import AutoTokenizer
from optimum.intel import OVModelForSeq2SeqLM
# from optimum.intel.openvino import OVQuantizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("pszemraj/flan-t5-large-grammar-synthesis")

# Load the model
model = OVModelForSeq2SeqLM.from_pretrained("pszemraj/flan-t5-large-grammar-synthesis", export=True)

# Function to correct grammar
def correct_grammar(text):
    input_ids = tokenizer(f"grammar: {text}", return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
text = "This sentences have bad grammer."
corrected_text = correct_grammar(text)
print(f"Original: {text}")
print(f"Corrected: {corrected_text}")

# # Optionally, you can quantize the model for faster inference
# quantizer = OVQuantizer.from_pretrained(model)
# quantized_model = quantizer.quantize(save_directory="quantized_model")

# # Use the quantized model
# quantized_model = OVModelForSeq2SeqLM.from_pretrained("quantized_model")
# outputs = quantized_model.generate(input_ids, max_new_tokens=100)
# quantized_corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(f"Quantized Corrected: {quantized_corrected_text}")

RuntimeError: Failed to import optimum.intel.openvino because of the following error (look up to see its traceback):
Failed to import optimum.onnx.graph_transformations because of the following error (look up to see its traceback):
DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.

In [2]:
from optimum.intel.openvino import OVQuantizer
# Optionally, you can quantize the model for faster inference
quantizer = OVQuantizer.from_pretrained(model)

quantized_model = quantizer.quantize_dynamic(
    save_dir="quantized_model_directory",
    # dataset="your_dataset_name",   # Optional: dataset for calibration if required
    # dataset_split="train",         # Optional: specify dataset split if needed
)

# quantized_model = quantizer.quantize(save_directory="quantized_model")

# # Use the quantized model
# quantized_model = OVModelForSeq2SeqLM.from_pretrained("quantized_model")
# outputs = quantized_model.generate(input_ids, max_new_tokens=100)
# quantized_corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(f"Quantized Corrected: {quantized_corrected_text}")

ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.

In [5]:
from optimum.intel import OVModelForSequenceClassification
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the model optimized for Intel hardware
model = OVModelForSequenceClassification.from_pretrained("bert-base-uncased", export=True)


RuntimeError: Failed to import optimum.intel.openvino because of the following error (look up to see its traceback):
DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.